<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/3/39/VU_logo.png/800px-VU_logo.png?20161029201021" width="350">

### Programming large-scale parallel systems


# Asynchronous programming in Julia


## Contents

In this notebook, we will learn the basics of asynchronous programming in Julia. In particular, we will learn about:

- Tasks
- Channels

Understanding these concepts is important to learn distributed computing later.

## Tasks

### Creating  a task

Technically, a task in Julia is a *symmetric co-routine*. More informally, a task is a piece of computation work that can be started (scheduled) at some point in the future, and that can be interrupted and resumed.  To create a task, we first need to create a function that represents the work to be done in the task. In next cell, we generate a task that generates and sums two matrices.

In [1]:
function work()
    println("Starting work")
    sleep(7)
    a = rand(3,3)
    b = rand(3,3)
    r = a + b
    println("Finishing work")
    r
end

work (generic function with 1 method)

In [3]:
t = Task(work)

Task (runnable) @0x00007fa4a4920e20

### Scheduling a task

The task has been created, but the corresponding work has not started. Note that we do not see any output from function `work` yet. To run the task we need to schedule it.

In [4]:
schedule(t)

Starting work


Task (runnable) @0x00007fa4a4920e20

Finishing work


### Fetching the task result

The task has been executed, but we do not see the result. To get the result we need to fetch it.

In [5]:
fetch(t)

3×3 Matrix{Float64}:
 1.07685   0.878607  0.376988
 0.942935  0.620054  0.684954
 1.33316   0.459834  1.0648

### Tasks run asynchronously

It is important to note that tasks run asynchronously. To illustrate this let's create and schedule a new task.

In [8]:
t = Task(work)

Task (runnable) @0x00007fa4a548dc30

In [9]:
schedule(t)

Starting work


Task (runnable) @0x00007fa4a548dc30

Note that while the task is running we can execute Julia code. To check this, execute the next two cells while the task is running.

In [57]:
sin(4π)*exp(-0.1)


-4.4324249909859736e-16

In [58]:
1 + 1

2

How is this possible? Tasks run in the background and this particular task is sleeping for most of the time. Thus, it is possible to use the current Julia process for other operations while the task is sleeping.

### Tasks do not run in parallel

It is also important to note that tasks do not run in parallel. We were able to run code while previous tasks was running because the task was idling most of the time in the sleep function.  If the task does actual work, the current process will be busy running this task and preventing to run other tasks. Let's illustrate this with an example. The following code computes an approximation of $\pi$ using [Leibniz formula](https://en.wikipedia.org/wiki/Leibniz_formula_for_pi). The quality of the approximation increases with the value of `n`.

In [5]:
function compute_π(n)
    s = 1.0
    for i in 1:n
        s += (isodd(i) ? -1 : 1) / (i*2+1)
    end
    4*s
end

compute_π (generic function with 1 method)

 Call this function with a large number. Note that it will take some time.

In [60]:
compute_π(4_000_000_000)

3.1415926538379395

Create a task that performs this computation.

In [61]:
fun = () -> compute_π(4_000_000_000)
t = Task(fun)

Task (runnable) @0x00007fa4a48e52d0

Schedule the tasks and then try to execute the 2nd cell bellow. Note that the current process will be busy running the task.

In [62]:
schedule(t)

Task (done) @0x00007fa4a48e52d0

In [63]:
1+1

2

### `yield`

If tasks do not run in parallel, what is the purpose of tasks? Tasks are handy since they can be interrupted and to switch control to other tasks. This is achieved via function `yield`. When we call yield, we provide the opportunity to switch to another task. The function below is a variation of function `compute_π` in which we yield every 1000 iterations. At the call to yield we allow other tasks to take over. Without this call to yield, once we start function `compute_π` we cannot start any other tasks until this function finishes.

In [27]:
function compute_π_yield(n)
    s = 1.0
    for i in 1:n
        s += (isodd(i) ? -1 : 1) / (i*2+1)
        if mod(i,1000) == 0
            yield()
        end
    end
    4*s
end

compute_π_yield (generic function with 1 method)

You can check this behavior experimentally with the two following cells. The next one creates and schedules a task that computes pi with the function `compute_π_yield`. Note that you can run the 2nd cell bellow while this task is running since we call to yield often inside  `compute_π_yield`.

In [28]:
fun = () -> compute_π_yield(3_000_000_000)
t = Task(fun)
schedule(t)

Task (runnable) @0x00007fa4a559c330

In [29]:
1+1

2

### Example: Implementing function sleep

Using yield, we can implement our own version of the sleep function as follows:

In [37]:
function mysleep(secs)
    final_time = time() + secs
    while time() < final_time
        yield()
    end
    nothing
end

mysleep (generic function with 1 method)

You can check that it behaves as expected.

In [38]:
@time mysleep(10)

 10.000001 seconds (209 allocations: 5.797 KiB)


### Tasks take a function with no arguments

This function needs to have zero arguments, but it can capture variables if needed. If we try to create a task with a function that has arguments, it will result in an error when we schedule it.

In [39]:
add(a,b) = a + b

add (generic function with 1 method)

In [40]:
t = Task(add)

Task (runnable) @0x00007fa4a48e60e0

In [41]:
schedule(t)

Task (failed) @0x00007fa4a48e60e0
MethodError: no method matching add()

Closest candidates are:
  add(::Any, ::Any)
   @ Main In[39]:1


If we need, we can capture variables in the function to be run by the task as shown in the next cells.

In [42]:
a = rand(3,3)
b = rand(3,3);

In [43]:
fun = () -> a + b
t = Task(fun)
schedule(t)

Task (done) @0x00007fa4a47c3080

### Useful macro: `@async`

So far, we have created tasks using low-level functions, but there are more convenient ways of creating and scheduling tasks. For instance using the `@async` macro. This macro is used to run a piece of code asynchronously. Under the hood it puts the code in an anonymous function, creates a task, and schedules it. For instance, the next cell is equivalent to the previous one.

In [44]:
@async a + b

Task (done) @0x00007fa4a47c3b70

### Another useful macro: `@sync`

This macro is used to wait for all the tasks created with `@async` in a given block of code. 

In [46]:
@sync begin
    @async sleep(3)
    @async sleep(4)
end

Task (done) @0x00007fa4a48241a0

## Channels

### Sending data between tasks

Julia provides channels as a way to send data between tasks. A channel is like a FIFO queue which tasks can put values into and take values from. In the next example, we create a channel and a task that puts five values into the channel. Finally, the task closes the channel.

In [47]:
chnl = Channel{Int}()

Channel{Int64}(0) (empty)

In [64]:
@async begin
    for i in 1:5
        put!(chnl,i)
    end
    close(chnl)
end

Task (runnable) @0x00007fa4a48e7530

By executing next cell several times, we will get the values from the channel. We are indeed communicating values from two different tasks. If we execute the cell more than 5 times, it will raise an error since the channel is closed. 

In [65]:
take!(chnl)

1

### Channels are iterable

Instead of taking values from a channel until an error occurs, we can also iterate over the channel in a for loop until the channel is closed.

In [66]:
chnl = Channel{Int}()

Channel{Int64}(0) (empty)

In [67]:
@async begin
    for i in 1:5
        put!(chnl,i)
    end
    close(chnl)
end

Task (runnable) @0x00007fa4a49004c0

In [68]:
for i in chnl
    @show i
end

i = 1
i = 2
i = 3
i = 4
i = 5


### Calls to `put!` and `take!` are blocking

Note that `put!` and `take!` are blocking operations. Calling `put!` blocks the tasks until another task calls `take!` and viceversa. Thus, we need at least 2 tasks for this to work. If we call `put!` and `take!` from the same task, it will result in a dead lock.  We have added a print statement to the previous example. Run it again and note how `put!` blocks until we call `take!`.  

In [89]:
chnl = Channel{Int}()

Channel{Int64}(0) (empty)

In [90]:
@async begin
    for i in 1:5
        put!(chnl,i)
        println("I have put $i")
    end
    close(chnl)
end

Task (runnable) @0x00007fa4a50d0330

In [91]:
take!(chnl)

I have put 1


1

### Buffered channels

We can be a bit more flexible and use a buffered channel. In this case, `put!` will block only if the channel is full and `take!` will block if the channel is empty. We repeat the previous example, but with a buffered channel of size 2. Note that we can call `put!` until the channel is full. At this point, we need to wait to until we call `take!` which removes an item from the channel, making room for a new item.

In [1]:
buffer_size = 2
chnl = Channel{Int}(buffer_size)

Channel{Int64}(2) (empty)

In [2]:
@async begin
    for i in 1:5
        put!(chnl,i)
        println("I have put $i")
    end
    close(chnl)
end

I have put 1


Task (runnable) @0x00007f23a75636c0

I have put 2


In [3]:
take!(chnl)

I have put 3


1

## Questions

In the next questions, `t` is the value of the variable `t` defined in the next cell.

In [6]:
t = @elapsed compute_π(100_000_000)

0.088960304

<div class="alert alert-block alert-success">
<b>Question (NB2-Q1):</b>  How long will the compute time of next cell be? 
</div>

    a) 10*t
    b) t
    c) 0.1*t
    d) near 0*t 


In [7]:
@time for i in 1:10
    compute_π(100_000_000)
end

  0.855730 seconds


<div class="alert alert-block alert-success">
<b>Question (NB2-Q2):</b>  How long will the compute time of next cell be? 
</div>

    a) 10*t
    b) t
    c) 0.1*t
    d) near 0*t 



In [9]:
@time for i in 1:10
    @async compute_π(100_000_000)
end

  0.000125 seconds (78 allocations: 6.727 KiB)


<div class="alert alert-block alert-success">
<b>Question (NB2-Q3):</b>  How long will the compute time of next cell be? 
</div>

    a) 10*t
    b) t
    c) 0.1*t
    d) near 0*t 


In [10]:
@time @sync for i in 1:10
    @async compute_π(100_000_000)
end

  0.886323 seconds (3.44 k allocations: 225.790 KiB, 2.56% compilation time)


<div class="alert alert-block alert-success">
<b>Question (NB2-Q4):</b>  How long will the compute time of the 2nd cell be? 
</div>

    a) infinity
    b) 1 second
    c) near 0 seconds
    d) 3 seconds

In [11]:
buffer_size = 4
chnl = Channel{Int}(buffer_size)

Channel{Int64}(4) (empty)

In [12]:
@time begin
    put!(chnl,3)
    i = take!(chnl)
    sleep(i)
end

  3.000999 seconds (99 allocations: 2.750 KiB)


<div class="alert alert-block alert-success">
<b>Question (NB2-Q5):</b>  How long will the compute time of the 2nd cell be? 
</div>

    a) infinity
    b) 1 second
    c) near 0 seconds
    d) 3 seconds

In [13]:
chnl = Channel{Int}()

Channel{Int64}(0) (empty)

In [ ]:
@time begin
    put!(chnl,3)
    i = take!(chnl)
    sleep(i)
end

# License

This notebook is part of the course [Programming Large Scale Parallel Systems](https://www.francescverdugo.com/XM_40017/) at Vrije Universiteit Amsterdam and may be used under a [CC BY 4.0](https://creativecommons.org/licenses/by/4.0/) license.